In [3]:
#import required packages
#使用ib_insync是一个能比较简单的提取数据的包（相比于ib_api自带的功能）
import pandas as pd
from ib_insync import *
util.startLoop()

In [4]:
#establish connection with IB TWS
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=100)
 

<IB connected to 127.0.0.1:7496 clientId=100>

In [25]:
#因为我们现在使用的是demo mode，所以这里使用汇率作为取数据演示
contract = Forex('USDJPY')
bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='1 Y',
    barSizeSetting='1 day', whatToShow='MIDPOINT', useRTH=True)

# convert to pandas dataframe:
df = util.df(bars)
df.head()


,date,open,high,low,close,volume,average,barCount
0,2019-01-22,109.6625,109.6970,109.1465,109.3805,-1,-1.0,-1
1,2019-01-23,109.3625,109.9995,109.3235,109.6065,-1,-1.0,-1
2,2019-01-24,109.6125,109.8010,109.4215,109.6485,-1,-1.0,-1
3,2019-01-25,109.6250,109.9515,109.4560,109.5490,-1,-1.0,-1
4,2019-01-28,109.5675,109.5675,109.1620,109.3460,-1,-1.0,-1


In [5]:
#因为demo mode所以现在会出现报错，使用实盘账户的话，应该可以跑通
code = "PDD"
 
contract = Stock(code, exchange="SMART")
bars = ib.reqHistoricalData(contract, endDateTime='20190314 18:30:00', durationStr='1 D',
        barSizeSetting='30 secs', whatToShow='MIDPOINT', useRTH=True)
 
# convert to pandas dataframe:
df = util.df(bars)
print(df[['date', 'open', 'high', 'low', 'close']])


Error 162, reqId 105: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Stock(symbol='PDD', exchange='SMART')


TypeError: 'NoneType' object is not subscriptable

In [9]:
#这是一个使用ib_sync和ib_api 来实现下单的一个例子
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)
 
order = MarketOrder('BUY', 1)
 
trade = ib.placeOrder(contract, order)
 
print(trade)
 
ib.sleep(1)
print(trade.log)
 
limitOrder = LimitOrder('BUY', 1, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)
 
print(limitTrade)
 
ib.cancelOrder(limitOrder)


Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=110, clientId=100, action='BUY', totalQuantity=1), orderStatus=OrderStatus(orderId=110, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0, lastLiquidity=0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 1, 21, 17, 7, 20, 809067, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])
[TradeLogEntry(time=datetime.datetime(2020, 1, 21, 17, 7, 20, 809067, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2020, 1, 21, 17, 7, 21, 96564, tzinfo=datetime.timezone.utc), status='Submitted', message='')]
Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=111, clientId=100, action='BUY', totalQuantity=1, lmtPrice=0.05), orderStatus=OrderStatus(orderId=111, status='PendingCancel', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0, lastLiquidity=0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 1, 21, 17, 7, 21, 817868, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2020, 1, 21, 17, 7, 21, 818816, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

In [43]:
#尝试调取option数据
spx = Index('SPX')
ib.qualifyContracts(spx)
chains= ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20200220, 20200319, 20200416, 20200514, 20200...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 650..."
1,CBOE,416904,SPXW,100,"[20200121, 20200122, 20200124, 20200127, 20200...","[800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0,..."
2,SMART,416904,SPXW,100,"[20200121, 20200122, 20200124, 20200127, 20200...","[800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0,..."
3,CBOE,416904,SPX,100,"[20200220, 20200319, 20200416, 20200514, 20200...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 650..."


In [47]:
#由于是demo mode 所以报错了，TWS setup 文件里最后一个链接中有具体的demo
[ticker] = ib.reqTickers(spx)
ticker
spxValue = ticker.marketPrice()
spxValue

chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

Error 10168, reqId 234: Requested market data is not subscribed. Delayed market data is not enabled, contract: Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')


0